In [10]:
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

# 2. Policy 로드
inference.load_policy("./model_14999.pt")

# 3. Observation 업데이트
inference.update_observation(
    base_ang_vel=[0.1, -0.05, 0.02],
    base_rpy=[0.05, 0.1, 0.0],
    velocity_commands=[0.5, 0.0, 0.0],
    joint_pos=[0.0] * 12,
    joint_vel=[0.1] * 12,
    actions=[0.0] * 12,
    depth_image=depth_data
)

# 4. Action 계산
action = inference.step()

# 5. 로봇 제어
inference.apply_action()

Go1PolicyInference initialized with device: cpu
Configuration: CNN=True, RNN=False, History=9
Initializing Go1PolicyInference...
Actor MLP+CNN: ActorDepthCNN(
  (prop_mlp): Sequential(
    (0): Linear(in_features=480, out_features=512, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ELU(alpha=1.0)
  )
  (depth_backbone): DepthOnlyFCBackbone(
    (image_compression): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ELU(alpha=1.0)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): ELU(alpha=1.0)
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=768, out_features=256, bias=True)
      (8): ELU(alpha=1

RuntimeError: Error(s) in loading state_dict for ActorCriticDepthCNN:
	size mismatch for actor.prop_mlp.0.weight: copying a param with shape torch.Size([512, 450]) from checkpoint, the shape in current model is torch.Size([512, 480]).
	size mismatch for critic.0.weight: copying a param with shape torch.Size([512, 1218]) from checkpoint, the shape in current model is torch.Size([512, 1296]).

In [9]:
# 수정된 코드로 테스트
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

# 2. Policy 로드 (수정된 코드로)
inference.load_policy("./model_14999.pt")

print("모델 로딩 성공!")


Go1PolicyInference initialized with device: cpu
Configuration: CNN=True, RNN=False, History=9
Initializing Go1PolicyInference...
Actor MLP+CNN: ActorDepthCNN(
  (prop_mlp): Sequential(
    (0): Linear(in_features=480, out_features=512, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ELU(alpha=1.0)
  )
  (depth_backbone): DepthOnlyFCBackbone(
    (image_compression): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ELU(alpha=1.0)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): ELU(alpha=1.0)
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=768, out_features=256, bias=True)
      (8): ELU(alpha=1

RuntimeError: Error(s) in loading state_dict for ActorCriticDepthCNN:
	size mismatch for actor.prop_mlp.0.weight: copying a param with shape torch.Size([512, 450]) from checkpoint, the shape in current model is torch.Size([512, 480]).
	size mismatch for critic.0.weight: copying a param with shape torch.Size([512, 1218]) from checkpoint, the shape in current model is torch.Size([512, 1296]).

In [11]:
# 더미 데이터로 전체 파이프라인 테스트
import torch
import numpy as np

# 더미 depth 이미지 생성 (24x32)
depth_data = np.random.rand(24, 32).astype(np.float32)

# 3. Observation 업데이트
inference.update_observation(
    base_ang_vel=[0.1, -0.05, 0.02],
    base_rpy=[0.05, 0.1, 0.0],
    velocity_commands=[0.5, 0.0, 0.0],
    joint_pos=[0.0] * 12,
    joint_vel=[0.1] * 12,
    actions=[0.0] * 12,
    depth_image=depth_data
)

# 4. Action 계산
action = inference.step()
print(f"계산된 액션: {action}")

# 5. 로봇 제어 시뮬레이션
inference.apply_action()

print("전체 파이프라인 테스트 완료!")


ValueError: Expected proprio obs dim 48, got 45

In [ ]:
# 체크포인트 상세 분석
import torch

# 체크포인트 로드
checkpoint = torch.load("./model_14999.pt", map_location="cpu")

print("=== 체크포인트 키 목록 ===")
for key in checkpoint.keys():
    print(f"- {key}")

print("\n=== 모델 상태 딕셔너리 키 샘플 ===")
model_state = checkpoint["model_state_dict"]
for i, key in enumerate(model_state.keys()):
    if i < 10:  # 처음 10개만 출력
        print(f"- {key}: {model_state[key].shape}")
    elif i == 10:
        print("... (더 많은 키들)")
        break

print("\n=== 중요한 레이어들 분석 ===")
# Actor prop_mlp 첫 번째 레이어
if "actor.prop_mlp.0.weight" in model_state:
    prop_weight = model_state["actor.prop_mlp.0.weight"]
    print(f"Actor prop_mlp 첫 번째 레이어: {prop_weight.shape}")
    print(f"  입력 차원: {prop_weight.shape[1]}")
    print(f"  출력 차원: {prop_weight.shape[0]}")

# Critic 첫 번째 레이어
if "critic.0.weight" in model_state:
    critic_weight = model_state["critic.0.weight"]
    print(f"Critic 첫 번째 레이어: {critic_weight.shape}")
    print(f"  입력 차원: {critic_weight.shape[1]}")
    print(f"  출력 차원: {critic_weight.shape[0]}")

# Action head
if "actor.action_head.weight" in model_state:
    action_weight = model_state["actor.action_head.weight"]
    print(f"Actor action head: {action_weight.shape}")
    print(f"  입력 차원: {action_weight.shape[1]}")
    print(f"  출력 차원: {action_weight.shape[0]}")


In [ ]:
# 히스토리 길이 계산 로직 분석
print("=== 히스토리 길이 계산 분석 ===")

# 체크포인트에서 실제 값들
prop_mlp_input_size = 450
critic_input_size = 1218
num_proprio_obs = 48  # 기본 proprioceptive observation 차원

print(f"체크포인트에서:")
print(f"  prop_mlp_input_size: {prop_mlp_input_size}")
print(f"  critic_input_size: {critic_input_size}")
print(f"  num_proprio_obs: {num_proprio_obs}")

# 현재 계산 방식 (잘못된 방식)
current_history_length = (prop_mlp_input_size // num_proprio_obs) - 1
print(f"\n현재 계산 방식:")
print(f"  history_length = (prop_mlp_input_size // num_proprio_obs) - 1")
print(f"  history_length = ({prop_mlp_input_size} // {num_proprio_obs}) - 1 = {current_history_length}")

# 실제로는 어떻게 계산되어야 하는지 확인
print(f"\n실제 계산:")
print(f"  prop_mlp_input_size = {prop_mlp_input_size}")
print(f"  num_proprio_obs = {num_proprio_obs}")
print(f"  prop_mlp_input_size / num_proprio_obs = {prop_mlp_input_size / num_proprio_obs}")
print(f"  정수 나눗셈: {prop_mlp_input_size // num_proprio_obs}")
print(f"  나머지: {prop_mlp_input_size % num_proprio_obs}")

# legged-loco의 실제 계산 방식 확인
print(f"\nlegged-loco 방식:")
print(f"  num_actor_obs_prop = num_actor_obs_prop * (history_length + 1)")
print(f"  따라서: history_length = (num_actor_obs_prop / num_proprio_obs) - 1")
print(f"  하지만 실제로는: {prop_mlp_input_size} / {num_proprio_obs} = {prop_mlp_input_size / num_proprio_obs}")

# 정확한 히스토리 길이 계산
if prop_mlp_input_size % num_proprio_obs == 0:
    correct_history_length = (prop_mlp_input_size // num_proprio_obs) - 1
    print(f"\n정확한 히스토리 길이: {correct_history_length}")
else:
    print(f"\n⚠️  나머지가 있음! 정확한 계산이 필요함")
    print(f"  450 / 48 = 9.375")
    print(f"  이는 히스토리 길이가 정수가 아님을 의미")


In [ ]:
# 근본 원인 분석: 실제 학습 설정과 체크포인트 불일치
print("=== 근본 원인 분석 ===")

# 1. 학습 시 설정 (StudyNote.md에서 확인)
print("1. 학습 시 설정:")
print("   --history_length=9")
print("   --use_cnn")
print("   --enable_cameras")

# 2. 체크포인트에서 실제 값
print("\n2. 체크포인트에서 실제 값:")
print("   prop_mlp_input_size: 450")
print("   critic_input_size: 1218")

# 3. 계산 분석
print("\n3. 계산 분석:")
print("   만약 history_length=9라면:")
print("   num_actor_obs_prop = 48 * (9 + 1) = 48 * 10 = 480")
print("   하지만 실제로는 450")

print("\n   만약 history_length=8라면:")
print("   num_actor_obs_prop = 48 * (8 + 1) = 48 * 9 = 432")
print("   하지만 실제로는 450")

print("\n   실제 계산:")
print("   450 / 48 = 9.375")
print("   이는 정수가 아님!")

# 4. 가능한 원인들
print("\n4. 가능한 원인들:")
print("   a) 학습 시 실제로는 다른 history_length가 사용됨")
print("   b) proprioceptive observation 차원이 48이 아님")
print("   c) 히스토리 버퍼링 방식이 다름")
print("   d) 체크포인트가 다른 설정으로 학습됨")

# 5. 실제 proprioceptive observation 차원 확인
print("\n5. 실제 proprioceptive observation 차원 확인:")
print("   450을 나누어떨어지게 하는 수들:")
for i in range(1, 50):
    if 450 % i == 0:
        print(f"   {i}: 450 / {i} = {450 // i}")
        if 450 // i - 1 == 9:
            print(f"   → {i}개 차원, history_length=9일 때 가능!")

print("\n6. 결론:")
print("   체크포인트는 history_length=9로 학습되지 않았거나,")
print("   proprioceptive observation 차원이 48이 아닐 가능성이 높음")


In [ ]:
# 실제 학습 설정 확인
print("=== 실제 학습 설정 확인 ===")

# agent.yaml에서 확인한 설정
print("agent.yaml에서 확인한 설정:")
print("  history_length: 9")
print("  use_cnn: true")
print("  obs_depth_shape: [24, 32]")
print("  actor_hidden_dims: [512, 256, 128]")
print("  critic_hidden_dims: [512, 256, 128]")

# 하지만 체크포인트에서는 다른 값
print("\n체크포인트에서 실제 값:")
print("  prop_mlp_input_size: 450")
print("  critic_input_size: 1218")

# 이제 정확한 계산을 해보자
print("\n=== 정확한 계산 ===")

# 450을 나누어떨어지게 하는 수들 중에서 history_length=9가 가능한지 확인
print("450의 약수들:")
for i in range(1, 51):
    if 450 % i == 0:
        history_length = (450 // i) - 1
        print(f"  proprio_dim={i}: history_length={history_length}")
        if history_length == 9:
            print(f"    → 이게 정답! proprio_dim={i}, history_length=9")

# 1218도 확인해보자
print("\n1218의 약수들 (critic input):")
for i in range(1, 51):
    if 1218 % i == 0:
        print(f"  {i}: 1218 / {i} = {1218 // i}")

print("\n=== 결론 ===")
print("실제 학습된 모델은:")
print("  proprioceptive observation 차원: 45 (48이 아님!)")
print("  history_length: 9")
print("  따라서: 45 * (9 + 1) = 45 * 10 = 450")
print("  critic input: 1218 (정확한 계산 필요)")


In [ ]:
# 최종 해결책: 올바른 설정으로 네트워크 재생성
print("=== 최종 해결책 ===")

# 실제 학습된 모델의 정확한 설정
actual_proprio_dim = 45  # 450 / 10 = 45
actual_history_length = 9
actual_prop_mlp_input = 450  # 45 * 10 = 450

print(f"실제 학습된 모델 설정:")
print(f"  proprioceptive observation 차원: {actual_proprio_dim}")
print(f"  history_length: {actual_history_length}")
print(f"  prop_mlp_input_size: {actual_prop_mlp_input}")

# 우리 코드에서 수정해야 할 부분
print(f"\n우리 코드에서 수정해야 할 부분:")
print(f"  config.py의 num_proprio_obs: 48 → {actual_proprio_dim}")
print(f"  또는 체크포인트에서 자동으로 감지하도록 수정")

# utils.py의 get_model_info_from_checkpoint 함수 수정 필요
print(f"\nutils.py 수정 방향:")
print(f"  1. 체크포인트에서 prop_mlp_input_size 추출")
print(f"  2. 가능한 proprio_dim과 history_length 조합 계산")
print(f"  3. 가장 합리적인 조합 선택 (history_length=9 우선)")

# 실제 계산 예시
print(f"\n실제 계산 예시:")
print(f"  prop_mlp_input_size = {actual_prop_mlp_input}")
print(f"  가능한 조합들:")
for proprio_dim in range(1, 51):
    if actual_prop_mlp_input % proprio_dim == 0:
        history_length = (actual_prop_mlp_input // proprio_dim) - 1
        print(f"    proprio_dim={proprio_dim}, history_length={history_length}")
        if history_length == 9:
            print(f"      → 정답! proprio_dim={proprio_dim}")


In [ ]:
# 수정된 코드로 최종 테스트
print("=== 수정된 코드로 최종 테스트 ===")

# 기존 inference 객체 제거
if 'inference' in locals():
    del inference

# 새로운 inference 객체 생성
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

# 2. Policy 로드 (수정된 자동 감지 로직 사용)
print("\n=== Policy 로딩 (수정된 로직) ===")
inference.load_policy("./model_14999.pt")

print("\n✅ 모델 로딩 성공!")


In [ ]:
# 정확한 proprioceptive observation 차원 계산
print("=== 정확한 계산 ===")

# legged-loco Go1 vision 설정에서 확인한 관측값들
print("Proprioceptive observation 구성요소:")
print("  base_lin_vel: 3차원")
print("  base_ang_vel: 3차원") 
print("  projected_gravity: 3차원")
print("  base_rpy: 3차원")
print("  velocity_commands: 3차원")
print("  joint_pos: 12차원")
print("  joint_vel: 12차원")
print("  actions: 12차원")

# 정확한 계산
total_proprio_dim = 3 + 3 + 3 + 3 + 3 + 12 + 12 + 12
print(f"\n정확한 계산:")
print(f"  3 + 3 + 3 + 3 + 3 + 12 + 12 + 12 = {total_proprio_dim}")

print(f"\n우리 코드의 잘못된 계산:")
print(f"  base_ang_vel(3) + base_rpy(3) + velocity_commands(3) + joint_pos(12) + joint_vel(12) + actions(12)")
print(f"  = 3 + 3 + 3 + 12 + 12 + 12 = 45")
print(f"  하지만 주석에는 + depth_image(768) - depth_image(768) = 48 이라고 잘못 표기됨")

print(f"\n실제로는:")
print(f"  proprioceptive observation 차원: {total_proprio_dim}")
print(f"  history_length: 9")
print(f"  prop_mlp_input_size: {total_proprio_dim} * (9 + 1) = {total_proprio_dim * 10}")

print(f"\n체크포인트와 비교:")
print(f"  체크포인트 prop_mlp_input_size: 450")
print(f"  계산된 값: {total_proprio_dim * 10}")
print(f"  일치 여부: {450 == total_proprio_dim * 10}")


In [ ]:
# 다시 정확히 계산해보자
print("=== 다시 정확히 계산 ===")

# 체크포인트에서 실제 값
checkpoint_prop_mlp_input = 450
checkpoint_history_length = 9

print(f"체크포인트에서:")
print(f"  prop_mlp_input_size: {checkpoint_prop_mlp_input}")
print(f"  history_length: {checkpoint_history_length}")

# 역산으로 실제 proprioceptive observation 차원 계산
actual_proprio_dim = checkpoint_prop_mlp_input // (checkpoint_history_length + 1)
print(f"\n역산 계산:")
print(f"  actual_proprio_dim = {checkpoint_prop_mlp_input} // ({checkpoint_history_length} + 1)")
print(f"  actual_proprio_dim = {checkpoint_prop_mlp_input} // 10 = {actual_proprio_dim}")

print(f"\n실제 legged-loco에서 사용된 관측값들:")
print(f"  base_lin_vel: 3차원")
print(f"  base_ang_vel: 3차원") 
print(f"  projected_gravity: 3차원")
print(f"  base_rpy: 3차원")
print(f"  velocity_commands: 3차원")
print(f"  joint_pos: 12차원")
print(f"  joint_vel: 12차원")
print(f"  actions: 12차원")
print(f"  총합: 3+3+3+3+3+12+12+12 = 51차원")

print(f"\n하지만 실제로는 45차원이 사용됨!")
print(f"이는 일부 관측값이 제외되었거나 다른 구성일 가능성")

# 45차원이 되려면 어떤 조합이 가능한지 확인
print(f"\n45차원이 되려면:")
print(f"  3 + 3 + 3 + 3 + 3 + 12 + 12 + 6 = 45 (actions가 6차원?)")
print(f"  또는")
print(f"  3 + 3 + 3 + 3 + 3 + 12 + 12 + 6 = 45 (다른 조합)")

print(f"\n결론: 실제 학습된 모델은 45차원 proprioceptive observation 사용")


In [ ]:
# 수정된 config.py로 최종 테스트
print("=== 수정된 config.py로 최종 테스트 ===")

# 기존 inference 객체 제거
if 'inference' in locals():
    del inference

# 새로운 inference 객체 생성
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

print(f"현재 설정:")
print(f"  num_proprio_obs: {inference.config.num_proprio_obs}")
print(f"  history_length: {inference.config.history_length}")
print(f"  num_actor_obs_prop: {inference.config.num_actor_obs_prop}")
print(f"  total_policy_input_dim: {inference.config.total_policy_input_dim}")

# 2. Policy 로드
print("\n=== Policy 로딩 ===")
inference.load_policy("./model_14999.pt")

print("\n✅ 모델 로딩 성공!")


In [12]:
# 수정된 utils.py로 최종 테스트
print("=== 수정된 utils.py로 최종 테스트 ===")

# 기존 inference 객체 제거
if 'inference' in locals():
    del inference

# 새로운 inference 객체 생성
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

print(f"초기 설정:")
print(f"  num_proprio_obs: {inference.config.num_proprio_obs}")
print(f"  history_length: {inference.config.history_length}")
print(f"  num_actor_obs_prop: {inference.config.num_actor_obs_prop}")
print(f"  total_policy_input_dim: {inference.config.total_policy_input_dim}")

# 2. Policy 로드 (수정된 자동 감지 로직 사용)
print("\n=== Policy 로딩 (수정된 로직) ===")
inference.load_policy("./model_14999.pt")

print(f"\n최종 설정:")
print(f"  num_proprio_obs: {inference.config.num_proprio_obs}")
print(f"  history_length: {inference.config.history_length}")
print(f"  num_actor_obs_prop: {inference.config.num_actor_obs_prop}")
print(f"  total_policy_input_dim: {inference.config.total_policy_input_dim}")

print("\n✅ 모델 로딩 성공!")


=== 수정된 utils.py로 최종 테스트 ===
Go1PolicyInference initialized with device: cpu
Configuration: CNN=True, RNN=False, History=9
Initializing Go1PolicyInference...
Actor MLP+CNN: ActorDepthCNN(
  (prop_mlp): Sequential(
    (0): Linear(in_features=480, out_features=512, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ELU(alpha=1.0)
  )
  (depth_backbone): DepthOnlyFCBackbone(
    (image_compression): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ELU(alpha=1.0)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): ELU(alpha=1.0)
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=768, out_features=256, bias

RuntimeError: Error(s) in loading state_dict for ActorCriticDepthCNN:
	size mismatch for actor.prop_mlp.0.weight: copying a param with shape torch.Size([512, 450]) from checkpoint, the shape in current model is torch.Size([512, 480]).
	size mismatch for critic.0.weight: copying a param with shape torch.Size([512, 1218]) from checkpoint, the shape in current model is torch.Size([512, 1296]).

In [ ]:
# 전체 파이프라인 테스트
print("=== 전체 파이프라인 테스트 ===")

import torch
import numpy as np

# 더미 depth 이미지 생성 (24x32)
depth_data = np.random.rand(24, 32).astype(np.float32)

# 3. Observation 업데이트
print("Observation 업데이트 중...")
inference.update_observation(
    base_ang_vel=[0.1, -0.05, 0.02],
    base_rpy=[0.05, 0.1, 0.0],
    velocity_commands=[0.5, 0.0, 0.0],
    joint_pos=[0.0] * 12,
    joint_vel=[0.1] * 12,
    actions=[0.0] * 12,
    depth_image=depth_data
)

# 4. Action 계산
print("Action 계산 중...")
action = inference.step()
print(f"계산된 액션: {action}")

# 5. 로봇 제어 시뮬레이션
print("로봇 제어 시뮬레이션...")
inference.apply_action()

print("\n🎉 전체 파이프라인 테스트 완료!")


In [ ]:
# 최종 수정된 코드로 테스트
print("=== 최종 수정된 코드로 테스트 ===")

# 기존 inference 객체 제거
if 'inference' in locals():
    del inference

# 새로운 inference 객체 생성
from go1_sim2real import Go1PolicyInference

# 1. 시스템 초기화
inference = Go1PolicyInference(device="cpu")
inference.init()

print(f"초기 설정:")
print(f"  num_proprio_obs: {inference.config.num_proprio_obs}")
print(f"  history_length: {inference.config.history_length}")
print(f"  num_actor_obs_prop: {inference.config.num_actor_obs_prop}")
print(f"  total_policy_input_dim: {inference.config.total_policy_input_dim}")

# 2. Policy 로드 (체크포인트에서 감지된 설정 사용)
print("\n=== Policy 로딩 (체크포인트 기반 설정) ===")
inference.load_policy("./model_14999.pt")

print(f"\n최종 설정:")
print(f"  num_proprio_obs: {inference.config.num_proprio_obs}")
print(f"  history_length: {inference.config.history_length}")
print(f"  num_actor_obs_prop: {inference.config.num_actor_obs_prop}")
print(f"  total_policy_input_dim: {inference.config.total_policy_input_dim}")

print("\n✅ 모델 로딩 성공!")


In [ ]:
# 전체 파이프라인 최종 테스트
print("=== 전체 파이프라인 최종 테스트 ===")

import torch
import numpy as np

# 더미 depth 이미지 생성 (24x32)
depth_data = np.random.rand(24, 32).astype(np.float32)

# 3. Observation 업데이트
print("Observation 업데이트 중...")
inference.update_observation(
    base_ang_vel=[0.1, -0.05, 0.02],
    base_rpy=[0.05, 0.1, 0.0],
    velocity_commands=[0.5, 0.0, 0.0],
    joint_pos=[0.0] * 12,
    joint_vel=[0.1] * 12,
    actions=[0.0] * 12,
    depth_image=depth_data
)

# 4. Action 계산
print("Action 계산 중...")
action = inference.step()
print(f"계산된 액션: {action}")

# 5. 로봇 제어 시뮬레이션
print("로봇 제어 시뮬레이션...")
inference.apply_action()

print("\n🎉 전체 파이프라인 테스트 완료!")


In [ ]:
# 간단한 정책 테스트 스크립트 실행
print("=== 간단한 정책 테스트 ===")

# 간단한 스크립트 실행
exec(open('simple_policy_test.py').read())


In [ ]:
# 초간단 정책 테스트
print("=== 초간단 정책 테스트 ===")

# 초간단 스크립트 실행
exec(open('ultra_simple_policy.py').read())
